## Input layer 바꾸기 -> patch embedding to linear

In [1]:
!pip install easydict

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6361 sha256=488619f6087cb6d9a358dea508cf3e8ca1aab6b3e5aded296726d4065e113d9b
  Stored in directory: c:\users\dlfwn\appdata\local\pip\cache\wheels\5d\79\e4\4e55effe206295359b37e0f9db3e68a1197ba396682807dadb
Successfully built easydict


In [3]:
!pip install torchsummary

In [5]:
!pip install numpy

In [6]:
!pip install matplotlib

In [7]:
!pip install timm

In [8]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=9015b017456c1eab2f20ec05ac5a2686271fa8d4d64c083494cf63273e7bb337
  Stored in directory: c:\users\dlfwn\appdata\local\pip\cache\wheels\23\9d\42\5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [9]:
!pip install albumentations

  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [12]:
!pip install opencv-python

In [15]:
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [16]:
!pip install tqdm

In [18]:
from easydict import EasyDict
from torchsummary import summary

In [141]:
import timm 

_model = timm.create_model('vit_base_patch16_224', pretrained=True)

In [142]:
data = torch.randn([3,224,224])
# data.shape
summary(_model, (3,224,224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
          Identity-2             [-1, 196, 768]               0
        PatchEmbed-3             [-1, 196, 768]               0
           Dropout-4             [-1, 197, 768]               0
         LayerNorm-5             [-1, 197, 768]           1,536
            Linear-6            [-1, 197, 2304]       1,771,776
           Dropout-7         [-1, 12, 197, 197]               0
            Linear-8             [-1, 197, 768]         590,592
           Dropout-9             [-1, 197, 768]               0
        Attention-10             [-1, 197, 768]               0
         Identity-11             [-1, 197, 768]               0
        LayerNorm-12             [-1, 197, 768]           1,536
           Linear-13            [-1, 197, 3072]       2,362,368
             GELU-14            [-1, 19

In [164]:
import torch
import torch.nn as nn

class custom(nn.Module) :
    def __init__(self, out) :
        super(custom, self).__init__()
        self.out = out
        self.model = nn.Sequential(nn.Linear(777, 768), _model.blocks, _model.norm, _model.pre_logits, _model.head)
        self.linear = nn.Linear(1000, out)
        self.avgpool1d = nn.AdaptiveAvgPool1d((1))
        
        
    def forward(self, x) :
        x = self.model(x)
        print(x.shape)
        x = x.transpose(2,1)
        print(x.shape)
        x = self.avgpool1d(x)
        print(x.shape)
        x = x.squeeze(2)
        print(x.shape)
        x = self.linear(x)
        print(x.shape)
        return x
modelcus = custom(25)

In [165]:
summary(modelcus, (768, 777), device='cpu')

torch.Size([2, 768, 1000])
torch.Size([2, 1000, 768])
torch.Size([2, 1000, 1])
torch.Size([2, 1000])
torch.Size([2, 25])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1             [-1, 768, 768]         597,504
         LayerNorm-2             [-1, 768, 768]           1,536
            Linear-3            [-1, 768, 2304]       1,771,776
           Dropout-4         [-1, 12, 768, 768]               0
            Linear-5             [-1, 768, 768]         590,592
           Dropout-6             [-1, 768, 768]               0
         Attention-7             [-1, 768, 768]               0
          Identity-8             [-1, 768, 768]               0
         LayerNorm-9             [-1, 768, 768]           1,536
           Linear-10            [-1, 768, 3072]       2,362,368
             GELU-11            [-1, 768, 3072]               0
          Dropout-12            [-1, 768, 3072

In [167]:
data = torch.randn((1, 768, 777))
pred = modelcus(data)
display(pred.shape)
display(pred)


torch.Size([1, 768, 1000])
torch.Size([1, 1000, 768])
torch.Size([1, 1000, 1])
torch.Size([1, 1000])
torch.Size([1, 25])


torch.Size([1, 25])

tensor([[-0.3285,  0.5938,  0.9646,  0.9012,  0.3262,  1.2143,  0.5092, -0.6902,
         -0.4887,  0.2381,  1.1416, -0.4408, -0.3450,  0.9816, -0.2160,  0.2707,
          0.0627, -0.7833,  1.3075,  0.5925,  1.2748,  0.6680, -0.3639,  0.6835,
         -0.3724]], grad_fn=<AddmmBackward>)